Goal of this script is to obtain 
vertex, edge, face 


V, EV

In [1]:
pwd()


"/home/mjirik/projects/lario3d.jl/notebooks"

# Read obj file

In [2]:

f = open("../test/smallbb2.obj");
lines = readlines(f)

nvertices = 0
nfaces = 0
for line in lines
    if line[1] == 'v'
        nvertices = nvertices + 1
    elseif line[1] == 'f'
        nfaces = nfaces + 1
    end
end
    


In [3]:

vertices = Array(Float64, nvertices , 3)
faces = Array(Int64, nfaces, 3)
print(size(vertices))
ivertices = 1
ifaces = 1

for line in lines
    lsplit = split(line)
    if line[1] == 'v'
        for i=2:4
            vertices[ivertices, i - 1] = parse(Float64, lsplit[i])
        end
#         print(vertices[ivertices,:])
#         println(line)
#         println(vertices[1:10,:])
        
        ivertices = ivertices + 1
    elseif line[1] == 'f'
        for i=2:4
            faces[ifaces, i - 1] = parse(Int64, lsplit[i])
        end
        ifaces = ifaces + 1
    else
        println("Cannot parse: ", line)
    end
end


(1192,3)

# Check Julia

In [4]:
A = rand(4,3);

B = view(A, 1:3, 1:2);

for i in eachindex(B)
   @show i
end


i = CartesianIndex{2}((1,1))
i = CartesianIndex{2}((2,1))
i = CartesianIndex{2}((3,1))
i = CartesianIndex{2}((1,2))
i = CartesianIndex{2}((2,2))
i = CartesianIndex{2}((3,2))


In [5]:
dcta = Dict("a" => 1, "b" => 2, "c" => 3)
dcta["f"] = 13
dcta

Dict{String,Int64} with 4 entries:
  "f" => 13
  "c" => 3
  "b" => 2
  "a" => 1

In [6]:
(1 , 5)


A = [(1,5), (15,18), (17,16,"asd"), (18,1,1)]

4-element Array{Tuple{Int64,Int64,Vararg{Any,N}},1}:
 (1,5)        
 (15,18)      
 (17,16,"asd")
 (18,1,1)     

In [7]:
i = 3
sort([faces[i, 1],  faces[i, 2]])

2-element Array{Int64,1}:
  9
 10

## Vertices

In [8]:
V = vertices

1192×3 Array{Float64,2}:
 1.0  2.0  1.0
 2.0  2.0  1.0
 1.0  3.0  1.0
 2.0  3.0  1.0
 1.0  2.0  1.0
 2.0  2.0  1.0
 1.0  2.0  2.0
 2.0  2.0  2.0
 1.0  2.0  1.0
 1.0  3.0  1.0
 1.0  2.0  2.0
 1.0  3.0  2.0
 2.0  2.0  1.0
 ⋮            
 4.0  6.0  5.0
 5.0  6.0  5.0
 4.0  6.0  6.0
 5.0  6.0  6.0
 4.0  4.0  6.0
 5.0  4.0  6.0
 4.0  5.0  6.0
 5.0  5.0  6.0
 4.0  5.0  6.0
 5.0  5.0  6.0
 4.0  6.0  6.0
 5.0  6.0  6.0

## Edges

In [9]:
# Eset = Set()
Epl = []
EVpl = []
FEpl = Array(Int64, nfaces, 3)

for i = 1:nfaces
    
    push!(EVpl, sort([faces[i, 1], faces[i, 2]]))
    push!(EVpl, sort([faces[i, 2], faces[i, 3]]))
    push!(EVpl, sort([faces[i, 3], faces[i, 1]]))
    
    FEpl[i, :] = [(i-1)*3 + 1, (i-1)*3 + 2,(i-1)*3 + 3]
    
end

EVpl
# Epl = EVpl


894-element Array{Any,1}:
 [1,2]      
 [2,4]      
 [1,4]      
 [5,6]      
 [6,8]      
 [5,8]      
 [9,10]     
 [10,12]    
 [9,12]     
 [13,14]    
 [14,16]    
 [13,16]    
 [17,18]    
 ⋮          
 [1177,1178]
 [1178,1180]
 [1177,1180]
 [1181,1182]
 [1182,1184]
 [1181,1184]
 [1185,1186]
 [1186,1188]
 [1185,1188]
 [1189,1190]
 [1190,1192]
 [1189,1192]

## FElar

In [10]:
# nzind2sparse(nzind)

FElar = spzeros(nfaces, (3*(nfaces)))
for i = 1:nfaces
    FElar[i, (i-1)*3 + 1] = 1
    FElar[i, (i-1)*3 + 2] = 1
    FElar[i, (i-1)*3 + 3] = 1
end


In [11]:
FEpl

298×3 Array{Int64,2}:
   1    2    3
   4    5    6
   7    8    9
  10   11   12
  13   14   15
  16   17   18
  19   20   21
  22   23   24
  25   26   27
  28   29   30
  31   32   33
  34   35   36
  37   38   39
   ⋮          
 859  860  861
 862  863  864
 865  866  867
 868  869  870
 871  872  873
 874  875  876
 877  878  879
 880  881  882
 883  884  885
 886  887  888
 889  890  891
 892  893  894

In [12]:
# EVpl = collect(Eset)

In [13]:
function arrayofarray2arrayd2d(d)
    data = Array(Float64,length(d),length(d[1]))
    for i in 1:length(d)
        for j in 1:length(d[1])
            data[i,j] = d[i][j]
        end
    end
    return data
end

arrayofarray2arrayd2d (generic function with 1 method)

## Clean vertexes

In [14]:
oo = collect(zip([1,2,3],[4,5,6]))

3-element Array{Tuple{Int64,Int64},1}:
 (1,4)
 (2,5)
 (3,6)

In [15]:
collect(zip([10,2,3], range(1,3)))

3-element Array{Tuple{Int64,Int64},1}:
 (10,1)
 (2,2) 
 (3,3) 

In [16]:
sort(collect(zip([10,2,3], range(1,3))))

3-element Array{Tuple{Int64,Int64},1}:
 (2,2) 
 (3,3) 
 (10,1)

In [17]:

function conv2array_of_float_arrays(array2d)

    test_vertex_float = float(array2d)

    list_of_vertexes = Array[]
    for i in 1:size(test_vertex_float, 1)
        push!(list_of_vertexes, test_vertex_float[i,:])
    end
    return list_of_vertexes
end
    

conv2array_of_float_arrays (generic function with 1 method)

In [18]:
function remove_double_vertexes_alternative(V)
    """ Return reduced vertex list wiht search table.
    
    Alternativte removing of doublefaces. Hopefully more memory efficient
    """
    X = range(1, size(V,1))
    # Vs = [v for (v, x) in VIsorted]
    # Is = [x for (v, x) in VIsorted]
    Vs = []
#     Is = [0]*size(V, 1)
#     Is = zeros( Int64, size(test_vertex, 1))
    
    Is = Dict()

    prevv = nothing
    i = 1
    sorted_vertex_id_x = sortperm(collect(zip(test_vertex[:,1], test_vertex[:,2])))
               sortperm(collect(zip(test_vertex[:,1], test_vertex[:,2])))
#     for couple in sort(collect(zip(X, conv2array_of_float_arrays(V))))
    for x in sorted_vertex_id_x
        v = V[x,:]
#         x = 
#         x, v = couple
        println(v, prevv, v.==prevv)
        if v == prevv
            # prev index was increased
            Is[x] = i - 1
        else
            push!(Vs, v)
            Is[x] = i
            i = i + 1
            prevv = v
        end
    end
    
    vertex_reduced_list = Vs
    vertex_search_table = Is
    return vertex_reduced_list, vertex_search_table
end

remove_double_vertexes_alternative (generic function with 1 method)

In [19]:

# sortperm(collect(zip(test_vertex[:,1], test_vertex[:,2])))

In [20]:
vtfl = conv2array_of_float_arrays(test_vertex)
vtfl

LoadError: UndefVarError: test_vertex not defined

In [21]:
[1 2 3 4] == [1 2 3 4]

true

In [22]:
dctd = Dict()
dctd["aho"] = 5
dctd["ahoi"] = 7

print(dctd)


Dict{Any,Any}(Pair{Any,Any}("aho",5),Pair{Any,Any}("ahoi",7))

In [23]:
sortperm([(6, 2), (2, 6), (1, 3)])

3-element Array{Int64,1}:
 3
 2
 1

## Construct EVlar


In [24]:
nedges = size(EVpl, 1)

894

In [25]:

EVlar = spzeros(Int8, nedges , nvertices )

for i = 1:nedges
    EVlar[i, EVpl[i][1]] = 1
    EVlar[i, EVpl[i][2]] = -1
end

EVlar


894×1192 sparse matrix with 1788 Int8 nonzero entries:
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
	[4   ,    5]  =  1
	[6   ,    5]  =  1
	[4   ,    6]  =  -1
	[5   ,    6]  =  1
	⋮
	[891 , 1185]  =  1
	[889 , 1186]  =  -1
	[890 , 1186]  =  1
	[890 , 1188]  =  -1
	[891 , 1188]  =  -1
	[892 , 1189]  =  1
	[894 , 1189]  =  1
	[892 , 1190]  =  -1
	[893 , 1190]  =  1
	[893 , 1192]  =  -1
	[894 , 1192]  =  -1

In [26]:
u = [1 1; 2 2]

2×2 Array{Int64,2}:
 1  1
 2  2

# FE

In [27]:
faces[1,2]

2

In [28]:

FE = spzeros(Int8, nfaces, nvertices)
for i = 1:nfaces
    FE[i, faces[i, 1]] = 1
    FE[i, faces[i, 2]] = 1
    FE[i, faces[i, 3]] = 1
end

In [29]:
vertices[7,:]
minimum(vertices)

0.0

In [30]:

include("../src/plasm.jl")

visualize_boundary (generic function with 1 method)

In [47]:

visualize_numbers([vertices, EVlar, FElar], 2)

1
ve while[1,2][1]2
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
wh 1
wh 2[2]
wh 3
wh 5
ve while[2,3][1,2]4
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
wh 1
wh 2[3]
wh 3
wh 5
ve while[2,3][1,2,3]4
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
wh 1
wh 2Int64[]


LoadError: BoundsError: attempt to access 0-element Array{Int64,1} at index [1]

# Debug version of visualize_numbers

In [55]:
function lar2plasm(EV, FE)
    EV = sparse(EV)
    faces = Array{Array{Int64, 1}, 1}()

    for f in 1:size(FE, 1)

        verts = Array{Int64, 1}()
        done = Array{Int64, 1}()
        face = FE[f, :]
        edges = EV[face.nzind, :]

        push!(done, 1)
        vs = edges[1, :].nzind
        if face.nzval[1] < 0
            vs = vs[end:-1:1]
        end
        startv, nextv = vs

        println("startv ", startv)
        push!(verts, startv)
        while nextv != startv
            println("ve while", edges[:, nextv].nzind, done, nextv, edges)
            ednz = edges[:, nextv].nzind
            println("wh 1")
            eall = setdiff(ednz, done)
            println("wh 2", eall)
            e = eall[1]
            println("wh 3")
            push!(done, e)
            vs = edges[e, :].nzind
            if face.nzval[e] < 0
                vs = vs[end:-1:1]
            end
            curv, nextv = vs
            push!(verts, curv)
            println("wh 5 ", nextv)
        end

        push!(faces, verts)
    end

    "FV = " * string(faces)[15:end]
end

function lar2py(V, EV)
    text = "V = [[0,0]"
    for i in 1:size(V, 1)
        text = text * "," * string(vec(V[i, :]))
    end
    text = text * "]\n"
    
    text = text * "EV = ["
    for i in 1:size(EV, 1)
        if i > 1 text = text * "," end
        text = text * string(EV[i, :].nzind)
    end
    text = text * "]"
    return text
end


function visualize_numbers(set, font_size)
    """
    seg = [V, EV, FE]
    """

    f = open("tmp.py", "w")

    file_content = "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size))\n"
    print(file_content)
    write(f, file_content)

    close(f)
#     run(`python tmp.py`)
end

function visualize_boundary(set, boundary, font_size)
    f = open("tmp.py", "w")
    file_content =  "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "model = larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size)\n" *
        lar2py(set[1], set[2][boundary.nzind, :]) * "\n" *
        "overmodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(STRUCT([model, COLOR(RED)(overmodel)]))"
    print(file_content)
    write(f, file_content)

    close(f)
    run(`python tmp.py`)
end

visualize_boundary (generic function with 1 method)

In [56]:
test_V = [
    1 1
    2 1
    1 2
    2 2
    ]
test_EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])
test_FE = sparse([
        1 1 -1
        -1 -1 1
])
visualize_numbers([test_V, test_EV, test_FE], 2)

startv 1
ve while[1,2][1]2
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5 3
ve while[2,3][1,2]3
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5 1
startv 2
ve while[1,3][1]1
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5 3
ve while[2,3][1,3]3
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5 2
from larlib import *
V = [[0,0],[1,1],[2,1],[1,2],[2,2]]
EV = [[1,2],[2,3],[1,3]]
FV = [[1,2,3],[2,1,3]]
VV = AA(LIST)(range(len(V)))
submodel = STRUCT(MKPOLS((V,EV)))
VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,2))


In [33]:
function printarr(arr)
    print(size(arr), typeof(arr), " sparse: ", issparse(arr),", min: ",  minimum(arr), ", max: ",  maximum(arr))
end

printarr (generic function with 1 method)

In [34]:
nvertices

1192

In [35]:
nfaces

298

In [36]:
printarr(vertices)

(1192,3)Array{Float64,2} sparse: false, min: 0.0, max: 6.0

In [45]:
printarr(EVlar)
# EVlar.nzind

(894,1192)SparseMatrixCSC{Int8,Int64} sparse: true, min: -1, max: 1

In [46]:
printarr(FElar)

(298,894)SparseMatrixCSC{Float64,Int64} sparse: true, min: 0.0, max: 1.0

In [57]:

visualize_numbers([vertices, EVlar, FElar], 2)

startv 1
ve while[1,2][1]2
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
wh 1
wh 2[2]
wh 3
wh 5 4
ve while[2,3][1,2]4
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
wh 1
wh 2[3]
wh 3
wh 5 4
ve while[2,3][1,2,3]4
	[1   ,    1]  =  1
	[3   ,    1]  =  1
	[1   ,    2]  =  -1
	[2   ,    2]  =  1
	[2   ,    4]  =  -1
	[3   ,    4]  =  -1
wh 1
wh 2Int64[]


LoadError: BoundsError: attempt to access 0-element Array{Int64,1} at index [1]

In [40]:
setdiff([2,3],[1,2,3])

0-element Array{Int64,1}

In [41]:
V = [
    1 1
    2 1
    1 2
    2 2
    ]
EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])
FE = sparse([
        1 1 -1
        -1 -1 1
])
visualize_numbers([V, EV, FE], 2)

1
ve while[1,2][1]2
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5
ve while[2,3][1,2]3
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5
2
ve while[1,3][1]1
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5
ve while[2,3][1,3]3
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5
from larlib import *
V = [[0,0],[1,1],[2,1],[1,2],[2,2]]
EV = [[1,2],[2,3],[1,3]]
FV = [[1,2,3],[2,1,3]]
VV = AA(LIST)(range(len(V)))
submodel = STRUCT(MKPOLS((V,EV)))
VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,2))


# Test visualization using LAR and Plasm

In [42]:
V = [
    1 1
    2 1
    1 2
    2 2
    ]
EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])
FE = sparse([
        1 1 -1
        -1 -1 1
])

2×3 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[2, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[1, 3]  =  -1
	[2, 3]  =  1

In [43]:
#visualize_numbers([V, EV, FE], 2)

In [167]:
function reindexVertexesInFaces(faces, new_indexes,
                                       index_base=0)
#     println(faces)
    for i in 1:size(faces, 1)
        for j in 1:size(faces, 2)
            faces[i, j] = new_indexes[faces[i, j] - index_base] + index_base
        end
    end
#     try
#     catch
#         println("reindexVertexInFaces exception")
#         println("fc ", face, " i ", i , " len ", size(new_indexes))
        
#     end
    return faces
    
end
    
        
#     for face in faces:
#         try:
#             for i in range(0, len(face)):
#                 face[i] = new_indexes[face[i] - index_base] + index_base
#             # face[0] = new_indexes[face[0]-1] + 1
#             # face[1] = new_indexes[face[1]-1] + 1
#             # face[2] = new_indexes[face[2]-1] + 1
#         except:
#             import traceback
#             traceback.print_exc()
#             print 'fc ', face, ' i ', i
#             print len(new_indexes)

#     return faces 
function removeDoubleFacesByDictAlgoritm(FW, sort_faces=true)
#     FWdict = Dict()
    set = Set()
    for k in 1:size(FW, 1)
        println(k)
        if sort_faces
            face = tuple(sort(FW[k, :])...)
        else
            face = tuple(FW[k, :]...)
        end
            
        push!(set, face)
    end
    return set
end
    

# def removeDoubleFacesByAlberto(FW):
#     from collections import defaultdict
#     # use of Albertos algorithm

#     cellDict = defaultdict(list)
#     for k, cell in enumerate(FW):
#         cellDict[tuple(cell)] += [k]
#     FW = [list(key) for key in cellDict.keys() if len(cellDict[key]) == 1]
#     return FW
function removeDoubleVertexesAndFaces(vertexes, faces, index_base=0)
    println("step1")
    new_vertexes, inv_vertexes = remove_double_vertexes_alternative(vertexes)# , index_base=index_base)
    println("step2")
    new_faces = reindexVertexesInFaces(faces, inv_vertexes)
    println("step3")
    new_faces = removeDoubleFacesByDictAlgoritm(faces)
    return new_vertexes, new_faces
end

removeDoubleVertexesAndFaces (generic function with 3 methods)

In [132]:
Dict(tuple(1,2) => 1, "B" => 2)

Dict{Any,Int64} with 2 entries:
  "B"   => 2
  (1,2) => 1

# Unittest

In [146]:
# test
test_vertex = [ 1 2; 2 3; 4 1; 2 3; 3 3]
test_faces = [
    1 3 4;
    4 5 1;
    1 5 4
]
# test_EV = sparse([
#     1 -1 0 0
#     0 1 -1 0
#     -1 0 1 0
# ])
# test_FE = sparse([
#         1 1 -1
#         -1 -1 1
# ])
# test_faces = sparse([ 1 2; 2 3; 4 1; 2 3])

Vs, Is = remove_double_vertexes_alternative(test_vertex)

for i in 1:size(test_vertex, 1)
    v1 = Vs[Is[i]] 
    v2 = test_vertex[i,:]
    assert(v1 == v2)
end

print(Vs)

[1,2]nothingBool[false,false]
[2,3][1,2]Bool[false,false]
[2,3][2,3]Bool[true,true]
[3,3][2,3]Bool[false,true]
[4,1][3,3]Bool[false,false]
Any[[1,2],[2,3],[3,3],[4,1]]

In [122]:

reindexed_faces = reindexVertexesInFaces(copy(test_faces), Is)
println(Vs)
println(reindexed_faces)
reindexed_faces == [1 4 2; 2 3 1]


Any[[1,2],[2,3],[3,3],[4,1]]
[1 4 2; 2 3 1]


true

In [153]:
fcs = removeDoubleFacesByDictAlgoritm(test_faces)

collect(fcs)

1
2
3


2-element Array{Any,1}:
 (1,4,5)
 (1,3,4)

In [169]:

new_vertexes, new_faces = removeDoubleVertexesAndFaces(test_vertex, test_faces)

step1
[1,2]nothingBool[false,false]
[2,3][1,2]Bool[false,false]
[2,3][2,3]Bool[true,true]
[3,3][2,3]Bool[false,true]
[4,1][3,3]Bool[false,false]
step2
step3
1
2
3


(Any[[1,2],[2,3],[3,3],[4,1]],Set(Any[(1,2,2),(1,2,4)]))